In [21]:
import os
cwd = os.getcwd()
if cwd[0:2] != "C:":
    from google.colab import drive
    drive.mount('/content/gdrive',force_remount=True)
    %cd "/content/gdrive/My Drive/Projects"
    !git clone https://github.com/ai357060/modelF.git
    import sys
    root_dir="/content/gdrive/My Drive/Projects/modelF"
    sys.path.append(root_dir)
    %cd "/content/gdrive/My Drive/Projects/modelF/wm"
    !git fetch --all
    !git reset --hard origin/master

In [22]:
import os
cwd = os.getcwd()
if cwd[0:2] != "C:":
    from google.colab import drive
    drive.mount('/content/gdrive',force_remount=True)
    %cd "/content/gdrive/My Drive/Projects"
#     !git clone https://github.com/ai357060/modelF.git
    import sys
    root_dir="/content/gdrive/My Drive/Projects/modelF"
    sys.path.append(root_dir)
    %cd "/content/gdrive/My Drive/Projects/modelF/wm"
#     !git pull

In [23]:
import os
import sys
module_path = os.path.abspath(os.path.join('..'))
if module_path not in sys.path:
    sys.path.append(module_path)

In [24]:
import pandas as pd
import numpy as np
import datetime as dt
import importlib
import tradestats
import pandas as pd
from datetime import datetime

In [25]:
from sklearn import datasets
from sklearn.tree import DecisionTreeRegressor
from sklearn.tree import DecisionTreeClassifier 
from sklearn import tree
import graphviz

In [26]:
importlib.reload(tradestats)
from tradestats import *

In [27]:
prices = loaddata_4h('AUDUSD_Candlestick_4_Hour_BID_03.08.2003-04.09.2021_EEF_GMT.csv')

In [28]:
rsiKey = [13]
tdiKey = [13]
dictList = []
colFeat = []
keyList = []
# rsiDict   = rsi(prices, rsiKey)
tdiDict   = tdi1(prices, tdiKey)
# dictList.append(rsiDict.df); colFeat.append('rsi'); keyList.append(rsiKey) 
dictList.append(tdiDict.df); colFeat.append('tdi'); keyList.append(tdiKey) 

In [29]:
# prices.dropna(inplace=True)
masterFrame = pd.DataFrame(index=prices.index)
masterFrame[['id','year','date','hour','weekday','open','high','low','close','volume']] = prices[['id','year','date','hour','weekday','open','high','low','close','volume']]   

In [30]:
for i in range(0,len(dictList)):#iteracja po funkcjach
    for j in keyList[i]: #iteracja po okresach
        for k in list(dictList[i][j]): #iteracja po kolumnach danej funkcji w okresie
            colID = colFeat[i] + str(j) + k
            masterFrame[colID] = dictList[i][j][k]

In [31]:
# masterFrame.to_csv(sep=';',path_or_buf='../Data/masterFrame.csv',date_format="%Y-%m-%d",index = False,na_rep='')

In [32]:
masterFrame = masterFrame.dropna()

In [33]:
# masterFrame.iloc[0:-50,:20]

In [34]:
importlib.reload(tradestats)
from tradestats import *

In [81]:
starttime = datetime.now()
# trades,alltrades = preparetrades(masterFrame,[-1,1],[5],[13],[0.003],False)
# trades,alltrades = preparetrades(masterFrame,[-1,1],[5,9],[13,17],[0.001,0.002,0.003,0.004,0.005,0.006],True)
alltrades = preparetrades(masterFrame,[1],[5],[13,17],[0.001,0.002,0.003,0.004,0.005,0.006])
trades = cleartrades(alltrades.copy(),False)
# trades,alltrades = preparetrades(masterFrame,[-1,1],[5],[13],[0.002],True)
endtime = datetime.now()
print(str(endtime - starttime))

0:00:10.366656


In [ ]:
#____________________________________________________________________________________________________________________
#
#
trades.head(2)

In [ ]:
X = trades.iloc[:, :-1].values
feature_names = trades.columns[:-1]
y = trades.iloc[:, -1:].values

In [ ]:
# regr = DecisionTreeRegressor(min_samples_leaf=0.001,criterion='mse',max_depth=17, random_state=1234)
# criterion{“mse”, “friedman_mse”, “mae”, “poisson”}, default=”mse”
regr = DecisionTreeClassifier(min_samples_leaf=1,criterion='gini',max_depth=8, random_state=1234)
# criterion{“gini”, “entropy”}, default=”gini”
regr.fit(X, y)
score = regr.score(X, y)
print(score)

In [ ]:
"""
all features mse depth 8 leaf 0.05 =0,0049
wyciąłem poniższe mse depth 8 leaf 0.05 =0,0043
#     df['green1_red1'] = df.green1 - df.red1
#     df['green2_red2'] = df.green2 - df.red2
#     df['green_red_change'] = df.green1_red1 - df.green2_red2
#     df['green_red_slope_change'] = df.green_slope - df.red_slope
all features mse depth 8 leaf 0.1 =0.0028
all features friedman_mse depth 8 leaf 0.05 =0.0049
all features mse depth 8 leaf 0.03 = 0.0069
all features mse depth 8 leaf 0.01 = 0.0074
all features mse depth 8 leaf 1 = 0.048
all features mse depth 9 leaf 1 = 0.051
all features mse depth 15 leaf 1 = 0.11
all features mse depth 15 leaf 0.001 = 0.054
all features mse depth 15 leaf 0.05 = 0,028
all features mse depth 5 leaf 0.05 = 0.0016
"""

In [ ]:
dd = pd.DataFrame()
dd['f'] = feature_names.values
dd['i'] = regr.feature_importances_
dd.sort_values(by=['i'], ascending=False)

In [ ]:
# text_representation = tree.export_text(regr)
# print(text_representation)

In [ ]:
from graphviz import Source
from sklearn import tree
graph = Source( tree.export_graphviz(regr, out_file=None, feature_names=feature_names,rounded = True, proportion = False, 
                 filled = True))
graph.format = 'png'
graph.render('dtree_render',view=True)

In [ ]:
alltrades.head()

In [ ]:
importlib.reload(tradestats)
from tradestats import *
stats = stattrades(alltrades)

In [ ]:
stats.sort_values("profit_ratio",ascending=False)

In [87]:
importlib.reload(tradestats)
from tradestats import *

tradetypes = [1]
openhours = [5]
closehours = [13]
sls = [10]
bar2froms = [-1000,-20,-10,0,10,20,1000]
bar2tos = [-1000,-20,-10,0,10,20,1000]
bar1froms = [-1000,-20,-10,0,10,20,1000]
bar1tos =[-1000,-20,-10,0,10,20,1000]
bar2froms = [-1000,1000]
bar2tos = [-1000,1000]
bar1froms = [-1000,1000]
bar1tos =[-1000,1000]
gr2froms = [-1000,-8,-3,0,3,8,1000]
gr2tos = [-1000,-8,-3,0,3,8,1000]
gr1froms = [-1000,-8,-3,0,3,8,1000]
gr1tos = [-1000,-8,-3,0,3,8,1000]
gr2froms = [-1000,-10,-8,-6,-4,-2,0,2,4,6,8,10,1000]
gr2tos = [-1000,-10,-8,-6,-4,-2,0,2,4,6,8,10,1000]
gr1froms = [-1000,-10,-8,-6,-4,-2,0,2,4,6,8,10,1000]
gr1tos = [-1000,-10,-8,-6,-4,-2,0,2,4,6,8,10,1000]
starttime = datetime.now()
stats = stathyper(alltrades,tradetypes,openhours,closehours,sls,bar2froms,bar2tos,bar1froms,bar1tos,gr2froms,gr2tos,gr1froms,gr1tos)
endtime = datetime.now()
print(str(endtime - starttime))

tradetype 1
  openhour 5
    closehour 13
      sl 10
        bar2from -1000
          bar2to 1000
            bar1from -1000
              bar1to 1000
            bar1from 1000
        bar2from 1000
0:01:38.623184


In [97]:
importlib.reload(tradestats)
from tradestats import *

tradetypes = [1]
openhours = [5]
closehours = [13]
sls = [10]
bar2froms = [-1000]
bar2tos = [1000]
bar1froms = [-1000]
bar1tos =[1000]

gr2froms = [-1000]
gr2tos = [1000]

gr1froms = [-1000,-6,-3,0,3,6,1000]
gr1tos = [-1000,-6,-3,0,3,6,1000]

grcfroms = [-1000,-6,-3,0,3,6,1000]
grctos = [-1000,-6,-3,0,3,6,1000]

rslopefroms = [-1000,-6,-4,-2,0,2,4,6,1000]
rslopetos = [-1000,-6,-4,-2,0,2,4,6,1000]

gslopefroms = [-1000,-10,-8,-6,-4,-2,0,2,4,6,8,10,1000]
gslopetos = [-1000,-10,-8,-6,-4,-2,0,2,4,6,8,10,1000]

starttime = datetime.now()
stats = stathyper(alltrades,tradetypes,openhours,closehours,sls,bar2froms,bar2tos,bar1froms,bar1tos,gr2froms,gr2tos,gr1froms,gr1tos,gslopefroms,gslopetos,rslopefroms,rslopetos,grcfroms,grctos)
endtime = datetime.now()
print(str(endtime - starttime))

tradetype 1
  openhour 5
    closehour 13
      sl 10
        bar2from -1000
          bar2to 1000
            bar1from -1000
              bar1to 1000
0:23:55.226900


In [94]:
stats.sort_values("profit_sum",ascending=False)

,tradetype,openhour,closehour,sl,bar2from,bar2to,bar1from,bar1to,gr2from,gr2to,...,countup,countdown,mean,profit_sum,maxdown,maxup,profit_ratio,maxdown_ratio,maxup_ratio,updown_ratio
0,1,5,13,10,-1000,1000,-1000,1000,-1000,1000,...,1585,1126,0.492032,1333.9,0,222.9,133.39,0,22.29,1.40764
0,1,5,13,10,-1000,1000,-1000,1000,-1000,1000,...,1549,1033,0.499729,1290.3,0,222.9,129.03,0,22.29,1.49951
0,1,5,13,10,-1000,1000,-1000,1000,-1000,1000,...,1583,1120,0.470699,1272.3,0,222.9,127.23,0,22.29,1.41339
0,1,5,13,10,-1000,1000,-1000,1000,-1000,1000,...,1597,1169,0.445300,1231.7,0,222.9,123.17,0,22.29,1.36612
0,1,5,13,10,-1000,1000,-1000,1000,-1000,1000,...,1478,855,0.526275,1227.8,0,222.9,122.78,0,22.29,1.72865
0,1,5,13,10,-1000,1000,-1000,1000,-1000,1000,...,1583,1125,0.453360,1227.7,0,222.9,122.77,0,22.29,1.40711
0,1,5,13,10,-1000,1000,-1000,1000,-1000,1000,...,1597,1171,0.437753,1211.7,0,222.9,121.17,0,22.29,1.36379
0,1,5,13,10,-1000,1000,-1000,1000,-1000,1000,...,2375,1470,0.307646,1182.9,0,222.9,118.29,0,22.29,1.61565
0,1,5,13,10,-1000,1000,-1000,1000,-1000,1000,...,1595,1163,0.424257,1170.1,0,222.9,117.01,0,22.29,1.37145
0,1,5,13,10,-1000,1000,-1000,1000,-1000,1000,...,1573,1108,0.436255,1169.6,0,222.9,116.96,0,22.29,1.41967


In [71]:
stats0.to_csv(sep=';',path_or_buf='../Data/stats0.csv',date_format="%Y-%m-%d",index = False,na_rep='')

In [70]:
stats0

,id,year,date,hour,weekday,open,high,low,close,volume,...,openprice,openindex,sl,closeindex,closeprice,closehour,slindex,slprice,profit,profit1
38,38,2003,2003-08-09 05:00:00,5,1,0.65209,0.65209,0.65209,0.65209,0.0000,...,0.65209,38.0,60.0,40.0,0.65209,13,-1.0,-1.00000,0.0,-1
44,44,2003,2003-08-10 05:00:00,5,1,0.65209,0.65209,0.65209,0.65209,0.0000,...,0.65209,44.0,60.0,46.0,0.65209,13,-1.0,-1.00000,0.0,-1
50,50,2003,2003-08-11 05:00:00,5,1,0.65282,0.65495,0.65222,0.65348,26580.2300,...,0.65282,50.0,60.0,52.0,0.65645,13,-1.0,-1.00000,36.3,1
56,56,2003,2003-08-12 05:00:00,5,1,0.65784,0.66100,0.65752,0.65875,23271.6202,...,0.65784,56.0,60.0,58.0,0.65679,13,-1.0,-1.00000,-10.5,-1
62,62,2003,2003-08-13 05:00:00,5,1,0.65442,0.65505,0.65201,0.65419,22689.6499,...,0.65442,62.0,60.0,64.0,0.65356,13,-1.0,-1.00000,-8.6,-1
68,68,2003,2003-08-14 05:00:00,5,1,0.65799,0.65895,0.65689,0.65842,24090.9703,...,0.65799,68.0,60.0,70.0,0.65633,13,-1.0,-1.00000,-16.6,-1
74,74,2003,2003-08-15 05:00:00,5,1,0.65688,0.65895,0.65641,0.65824,23279.3696,...,0.65688,74.0,60.0,76.0,0.65854,13,-1.0,-1.00000,16.6,-1
80,80,2003,2003-08-16 05:00:00,5,1,0.65951,0.65951,0.65951,0.65951,0.0000,...,0.65951,80.0,60.0,82.0,0.65951,13,-1.0,-1.00000,0.0,-1
86,86,2003,2003-08-17 05:00:00,5,1,0.65951,0.65951,0.65951,0.65951,0.0000,...,0.65951,86.0,60.0,88.0,0.65951,13,-1.0,-1.00000,0.0,-1
92,92,2003,2003-08-18 05:00:00,5,1,0.66066,0.66172,0.65985,0.66045,26031.6196,...,0.66066,92.0,60.0,94.0,0.65848,13,-1.0,-1.00000,-21.8,-1
